# Crawling Website Berita Online

# Install Library


In [ ]:
!pip install builtwith
!pip install requests
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd

  Preparing metadata (setup.py) ... done
  Created wheel for builtwith: filename=builtwith-1.3.4-py3-none-any.whl size=36077 sha256=56021b9df7e7607dc7c360490d2aa85ab28b5ad7f9df0829e9931b16cd027a69
  Stored in directory: /root/.cache/pip/wheels/7f/2d/b2/606e3df914d4aeeab99c4a4e3e9a61673d2293c2e346db00c8
Successfully built builtwith


# analisis teknologi yang digunakan dalam website

In [ ]:
import builtwith

# Analisis teknologi yang digunakan
res = builtwith.parse('https://www.antaranews.com/')
print(res)

{'databases': ['Firebase'], 'font-scripts': ['Google Font API'], 'web-frameworks': ['Twitter Bootstrap'], 'javascript-frameworks': ['jQuery']}


# Web Crawling

In [ ]:
import requests
from bs4 import BeautifulSoup

def crawl_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        # Ambil semua judul h1, h2, h3
        headings = soup.find_all(['h1', 'h2', 'h3'])
        for heading in headings:
            print(f"{heading.name}: {heading.get_text()}")

        # Ambil semua link
        links = soup.find_all('a', href=True)
        for link in links:
            print(f"URL: {link['href']} | Teks: {link.get_text()}")

    except requests.exceptions.RequestException as e:
        print(f"Terjadi kesalahan saat mengakses {url}: {e}")

# Gunakan fungsi
crawl_website("https://www.antaranews.com/terkini")

h1: Terkini
h2: Polisi: Freeport terus berjuang evakuasi pekerja terjebak di tambang
h2: KTT Sabuk dan Jalur Sutra di Hong Kong eksplorasi peluang kerja sama
h2: Komisi I DPR minta WNI segera dievakuasi dari Nepal imbas kerusuhan
h2: Pemprov Bali hitung kerugian 474 pedagang pasar terdampak banjir 
h2: Truk tangki gas meledak di Meksiko, sekitar 57 orang luka-luka
h2: KPK siap hadiri sidang praperadilan Rudy Tanoesoedibjo
h2: Trump disebut frustasi dengan Netanyahu atas serangannya ke Qatar
h2: Sekjen: Bonus demografi bisa tentukan capaian pertumbuhan ekonomi
h2: Pemerintah sambut komitmen Roblox lindungi anak di ruang digital
h2: Kerja keras mengakhiri kemiskinan ekstrem
h2: Kemendikdasmen bangun kelas darurat dukung KBM di SMKN 1 Cileungsi
h2: KPK tetapkan Rudy Tanoe sebagai tersangka kasus pengangkutan bansos
h2: Kemdiktisaintek gelontorkan dana Rp110 miliar untuk riset mahasiswa
h2: Dinsos Lampung: Sekolah Rakyat 1C segera mulai pembelajaran
h2: Pengamat: Indonesia butuh Menko Polk

# Web Scraping

In [ ]:
def crawl_berita_antaranews(rss_url="https://www.antaranews.com/rss/terkini.xml", timeout=10):
    """
    Crawl berita dari ANTARA News menggunakan RSS feed.
    Mengembalikan DataFrame dengan kolom:
    id_berita, judul_berita, isi_berita, kategori_berita
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
    }
    session = requests.Session()
    session.headers.update(headers)

    try:
        r = session.get(rss_url, timeout=timeout)
        r.raise_for_status()
    except Exception as e:
        print("Gagal mengakses RSS:", rss_url, "-", e)
        return pd.DataFrame()

    soup = BeautifulSoup(r.content, "xml")
    items = soup.find_all("item")
    data = []

    for item in items:
        link_tag = item.find("link")
        link = link_tag.get_text(strip=True) if link_tag else None
        judul = item.title.get_text(strip=True) if item.title else ""
        guid = item.guid.get_text(strip=True) if item.guid else (link or "")
        kategori = item.category.get_text(strip=True) if item.category else None

        isi = ""
        if not link:
            # kalau tidak ada link, skip
            data.append({
                "id_berita": guid,
                "judul_berita": judul,
                "isi_berita": isi,
                "kategori_berita": kategori or "Tidak Ada"
            })
            continue

        try:
            r2 = session.get(link, timeout=timeout)
            r2.raise_for_status()
            soup2 = BeautifulSoup(r2.content, "html.parser")

            # ambil isi (beberapa alternatif selector sebagai fallback)
            konten = (
                soup2.select_one("div.wrap__article-detail-content.post-content")
                or soup2.select_one("div.article-content")
                or soup2.select_one("div.article-body")
                or soup2.select_one("article")
                or soup2.select_one("p")
            )
            if konten:
                isi = konten.get_text(separator="\n", strip=True)

            # ambil kategori dari breadcrumbs (lebih andal untuk struktur yang kamu kirim)
            breadcrumbs = soup2.select("ul.breadcrumbs a.breadcrumbs__url")
            if breadcrumbs:
                # Struktur umum: [Home, Kategori, Judul Artikel]
                # Pilih item kedua jika ada, atau item -2 (jika last adalah judul)
                if len(breadcrumbs) >= 3:
                    kategori = breadcrumbs[-2].get_text(strip=True)
                elif len(breadcrumbs) >= 2:
                    # kalau hanya Home + Kategori, ambil index 1
                    kategori = breadcrumbs[1].get_text(strip=True)
                else:
                    kategori = breadcrumbs[0].get_text(strip=True)

        except Exception as e:
            # Jika ada error saat fetch halaman artikel, tetap simpan meta dari RSS
            print(f"Warning: gagal mengambil halaman artikel {link}: {e}")

        data.append({
            "id_berita": guid,
            "judul_berita": judul,
            "isi_berita": isi,
            "kategori_berita": kategori or "Tidak Ada"
        })

    df = pd.DataFrame(data)
    return df

# Contoh penggunaan:
df_berita = crawl_berita_antaranews()
print(df_berita[["judul_berita", "kategori_berita"]].head(10))
df_berita.to_csv("berita_antaranews.csv", index=False, encoding="utf-8-sig")
print("Berhasil mengambil", len(df_berita), "berita.")

                                        judul_berita kategori_berita
0  Polisi: Freeport terus berjuang evakuasi peker...       Humaniora
1  KTT Sabuk dan Jalur Sutra di Hong Kong eksplor...           Dunia
2  Komisi I DPR minta WNI segera dievakuasi dari ...         Politik
3  Pemprov Bali hitung kerugian 474 pedagang pasa...       Humaniora
4  Truk tangki gas meledak di Meksiko, sekitar 57...            Foto
5  KPK siap hadiri sidang praperadilan Rudy Tanoe...           Hukum
6  Trump disebut frustasi dengan Netanyahu atas s...   Internasional
7  Sekjen: Bonus demografi bisa tentukan capaian ...       Humaniora
8  Pemerintah sambut komitmen Roblox lindungi ana...           Tekno
9          Kerja keras mengakhiri kemiskinan ekstrem      Infografik
Berhasil mengambil 50 berita.


# Hasil Scraping Berita

In [ ]:
df = pd.read_csv("berita_antaranews.csv")
df.head()

,id_berita,judul_berita,isi_berita,kategori_berita
0,https://www.antaranews.com/berita/5101501/poli...,Polisi: Freeport terus berjuang evakuasi peker...,Timika (ANTARA) - Kepolisian Sektor Tembagapur...,Humaniora
1,https://www.antaranews.com/berita/5101497/ktt-...,KTT Sabuk dan Jalur Sutra di Hong Kong eksplor...,Hong Kong (ANTARA) - Konferensi Tingkat Tinggi...,Dunia
2,https://www.antaranews.com/berita/5101493/komi...,Komisi I DPR minta WNI segera dievakuasi dari ...,Jakarta (ANTARA) - Anggota Komisi I DPR RI Tau...,Politik
3,https://www.antaranews.com/berita/5101489/pemp...,Pemprov Bali hitung kerugian 474 pedagang pasa...,Denpasar (ANTARA) - Pemerintah Provinsi Bali t...,Humaniora
4,https://www.antaranews.com/foto/5101485/truk-t...,"Truk tangki gas meledak di Meksiko, sekitar 57...",Petugas pemadam kebakaran bekerja di lokasi ke...,Foto
